# E4D log file checker
This notebook will check the E4D logfile in the current directory and produce some Tikhonov plots and Chi2 plots. Click Kernal>Restart and Run all, and scroll down to the bottom to see the current state of the inversion. Re-Run the bottom cell to re-update and plot the data. 

To use:
Edit the filenames below for the files you want to process.
Filenames is a list of strings. It should be defined by something like:
> ['filename1', 'filename2', 'filename3']

then Kernal>Restart and Run All

Scroll to the bottom for the plots.

Sometimes Jupyter puts multiple plots into a single scroll windw output. Not sure on the precise criteria for this. Just make sure you dont scroll past the plot you want, check for scrollbars on the outputs.

In [ ]:
# EDIT THESE
filenames=['e4d.log','e4d_2.log','e4d_3.log', 'e4d_4.log']


In [ ]:
import matplotlib.pyplot as plt
import os, glob
%matplotlib inline

In [ ]:


files=[]
for filename in filenames:
    files.append(open(filename))

file_texts=[]
for file in files:
    file_texts.append(file.readlines())

#### Function Definitions

In [ ]:
def readint(textline,num2read):
    """
    Read an element from a line of text of format ' a b c d' in the num2read'th position
    returns it as an int
    """
    nums=0
    prev_char=' '
    zone=''
    for char in textline:
        if prev_char==' ' and char != ' ':
            nums+=1
        prev_char=char
        if nums==num2read and (char!=' ' ):
            zone=zone+char #concatenate
    return int(zone)

def readreal(textline,num2read):
    """
    Read an element from a line of text of format ' a b c d' in the num2read'th position
    returns it as a real
    """
    nums=0
    prev_char=' '
    zone=''
    for char in textline:
        if prev_char==' ' and char != ' ':
            nums+=1
        prev_char=char
        if nums==num2read and (char!=' '):
            zone=zone+char #concatenate
    return float(zone)

In [ ]:
#read through the text file, extracting data values if they are needed
def data_extract(text, printBool=False,n_constraints=1): 
    print('Length of text: ', len(text))
    if printBool:
        for i in range(10):
            print(text[i], end='')
    chi_list=[]
    phimodxbeta_list=[]
    phimod_list=[]
    misfit_list=[]
    cmplx_start=-1
    for i, line in enumerate(text):
        if(line[:23]==' ********** CONVERGENCE'):
            #find phi_mod, beta, and chi
            chi_list.append(readreal(text[i+8+(n_constraints-1)],4))
            phimod_list.append(readreal(text[i+2],3))
            phimodxbeta_list.append(readreal(text[i+2],2))
            misfit_list.append(readreal(text[i+2],1))
        if(line[:23]==' ********** FINAL SOLUT'):
            chi_list.append(readreal(text[i+3],4))
            phimod_list.append(readreal(text[i+2],3))
            phimodxbeta_list.append(readreal(text[i+2],2))
            misfit_list.append(readreal(text[i+2],1))
        if(line[:52]==' ********** CONVERGENCE STATISTICS AT STARTING MODEL'):
            if(cmplx_start==-1):
                cmplx_start=-2
            else:
                cmplx_start=len(phimod_list)-1
    print('Finished!')
    return phimod_list, misfit_list, chi_list, cmplx_start

In [ ]:
def tik_plot(phimod_list,misfit_list,plt_start, pltEnd=0, title='1'):
    if(pltEnd<=0):
        pltEnd=len(phimod_list)
    fig, ax1=plt.subplots()
    ax1.set_ylabel('mistfit')
    ax1.set_xlabel('phi_mod')
    ax1.loglog(phimod_list[plt_start:pltEnd],misfit_list[plt_start:pltEnd],'ro-')
    ax2=ax1.twinx()
    ax1.grid()
    ax2.plot(phimod_list[plt_start:pltEnd],list(range(len(phimod_list[plt_start:pltEnd]))),'bo')
    ax2.set_ylabel('Iteration')
    labels=['tik plot (misfit)','iteration graph']
    fig.legend(labels)
    ax1.set_title('Tikhonov regression plot ' + title)
    plt.show()
    return


def plt_chi2(chi2_list,plt_start, pltEnd=0, title='1'):
    if(pltEnd<=0):
        pltEnd=len(chi2_list)
    fig, ax1=plt.subplots()
    ax1.set_ylabel('chi2')
    ax1.set_xlabel('iteration')
    ax1.semilogy(list(range(len(chi2_list[plt_start:pltEnd]))),chi2_list[plt_start:pltEnd],'bo-')
    labels=['chi2 plot']
    fig.legend(labels)
    ax1.set_title('Chi2 plot ' + title)
    ax1.axhline(1, c='g', ls='--')
    ax1.grid()
    ax1.axhline(10, c='r', ls='--')
    labels=['chi2 plot']
    fig.legend(labels)
    plt.show()
    return

In [ ]:
# Find number of constraints
def find_constraints(text):
    """
    Finds the the number of constraints in a text block
    Input: text
    Output: constraint (int) the number of constraints in the file
    """
    for line in text:
        if(line[:30]=='  Number of constraint blocks:'):
            constraint= readint(line,5)
    return constraint

## Log File Information
This section of the notebook uses some of the functions above to get the data from the logfile


In [ ]:
constraints=[]
for text in file_texts:
    constraints.append(find_constraints(text))

phimodlists =[None]*len(files)
misfitlists =[None]*len(files)
chilists    =[None]*len(files)
cmplx_starts=[None]*len(files)



for i, text in enumerate(file_texts):
    phimodlists[i], misfitlists[i], chilists[i], cmplx_starts[i] = data_extract(text,n_constraints=constraints[i])


In [ ]:
cmplx_starts[0]

## This section actually creates the plots

##### Real inversions:

Tik Plots

In [ ]:
#Tik plot, match iter blue dot vertically with tik plot red dot for the iteration
# As long as misfit doesnt increase, the inversion can be said to be progressing in a way.
# Be careful of oscilations later on
for i in range(len(phimodlists)):
    tik_plot(phimodlists[i],misfitlists[i],0,cmplx_starts[i], title='- Conductivity,  '+filenames[i])

Chi2 Plots

In [ ]:
# Chi2 plots
for i in range(len(chilists)):
    plt_chi2(chilists[i],0,cmplx_starts[i], title='Conductivity, ' + filenames[i])

##### Phase inversions:

Tik:

In [ ]:

for i in range(len(chilists)):
    if(cmplx_starts[i]>0):
        tik_plot(phimodlists[i],misfitlists[i],cmplx_starts[i],0, title='- Phase,  '+filenames[i])
    else:
        print("   No complex values found for file ", filenames[i], " - continuing with next file if it exists")
        print(" ------------------------------------------------------------------")

Chi2:

In [ ]:

for i in range(len(chilists)):
    if(cmplx_starts[i]>0):
        plt_chi2(chilists[i],cmplx_starts[i],0, title='Phase, ' + filenames[i])
    else:
        print("   No complex values found for file ", filenames[i], " - continuing with next file if it exists")
        print(" ------------------------------------------------------------------")